<img src="../exploratoryAnalysis/zona_jobs.png">

# Trabajo Practico N° 2

### Aqui procederemos a la limpieza de datos

#### 1-1 Carga de datos

In [3]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')
import plotly

In [ ]:
# Carga de archivos
post_educacion  = pd.read_csv('../data/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [ ]:
post_gen_edad   = pd.read_csv('../data/fiuba_2_postulantes_genero_y_edad.csv', dtype = {
    'sexo': 'category'    
})

In [ ]:
vistas          = pd.read_csv('../data/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [ ]:
postulaciones   = pd.read_csv('../data/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])

In [4]:
avisos_online   = pd.read_csv('../data/fiuba_5_avisos_online.csv')

In [ ]:
avisos_detalles = pd.read_csv('../data/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [ ]:
# Carga de archivos
post_educacion_pre  = pd.read_csv('../data/pre_abril/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [ ]:
post_gen_edad_pre   = pd.read_csv('../data/pre_abril/fiuba_2_postulantes_genero_y_edad.csv', dtype = {
    'sexo': 'category'    
})

In [5]:
vistas_pre          = pd.read_csv('../data/pre_abril/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [ ]:
postulaciones_pre   = pd.read_csv('../data/pre_abril/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])

In [ ]:
avisos_online_pre   = pd.read_csv('../data/pre_abril/fiuba_5_avisos_online.csv')

In [ ]:
avisos_detalles_pre = pd.read_csv('../data/pre_abril/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [ ]:
# Carga de archivos
post_educacion_post  = pd.read_csv('../data/post_abril/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [ ]:
vistas_post          = pd.read_csv('../data/post_abril/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [ ]:
avisos_detalles_post = pd.read_csv('../data/post_abril/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

## Ahora agrupo todos en un mismo DF

In [ ]:
frames = [ post_educacion, post_educacion_pre, post_educacion_post ]
post_ed_total = pd.concat(frames)
post_ed_total.head()

In [ ]:
frames = [ post_gen_edad, post_gen_edad_pre]
post_gen_edad_total = pd.concat(frames)
post_gen_edad_total.head()

In [ ]:
frames = [ vistas_pre, vistas_post]
vistas_total = pd.concat(frames)
vistas_total.head()

In [ ]:
frames = [ postulaciones, postulaciones_pre]
postulaciones_total = pd.concat(frames)
postulaciones_total.head()

In [ ]:
frames = [ avisos_online, avisos_online_pre]
avisos_online_total = pd.concat(frames)
avisos_online_total.head()

In [ ]:
frames = [ avisos_detalles, avisos_detalles_pre, avisos_detalles_post]
avisos_detalles_total = pd.concat(frames)
avisos_detalles_total.head()

In [ ]:
# Recorto la columna idpais ya que no es necesaria.
avisos_detalles_total.drop(['idpais'] , axis=1, inplace=True)

In [ ]:
# Agrupo las áreas parecidas.
areas_agrupadas = pd.read_csv('../extraData/areas_agrupadas.csv')
avisos_detalles_total = pd.merge(avisos_detalles_total, areas_agrupadas, how='inner', on='nombre_area')
avisos_detalles_total['nombre_area'] = avisos_detalles_total['nuevo_nombre_area']
avisos_detalles_total.drop('nuevo_nombre_area', axis=1, inplace=True)
avisos_detalles_total.head()

In [ ]:
# Convierto el nombre del area a category.
avisos_detalles_total['nombre_area'] = avisos_detalles['nombre_area'].astype('category')

In [ ]:
# Agrego tipo categorico y modifico el tipo de dato de la fecha.
post_gen_edad_total.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad_total['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')

In [ ]:
# Renombramos algunas columnas para unificar criterios.
vistas_total.rename(columns={"idAviso":"idaviso"}     , inplace=True)
vistas_total.rename(columns={"timestamp":"fechavista"}, inplace=True)

# Cambiar el tipo de dato de la fecha de vista.
vistas_total[['fechavista']] = vistas_total[['fechavista']].apply(pd.to_datetime)

## Luego de mergear los df debo quitar cols

### Para poder extraer info innecesaria y poder utilizar en el entrenamiento

In [ ]:
# Asigno puntajes a los diferentes niveles educativos encontrados para poder 
# obtener asi la educacion mas valiosa (la de nivel academico mas alto) de cada postulante.
valores_educacion = pd.DataFrame({
    'nombre': [
        'Otro',  'Otro',  'Otro',  'Secundario',  'Secundario',  'Secundario',  'Terciario/Técnico', 'Universitario',  \
        'Terciario/Técnico', 'Universitario',  'Terciario/Técnico', 'Universitario',  'Posgrado', 'Master','Posgrado', \
        'Master','Posgrado', 'Master','Doctorado','Doctorado','Doctorado'
    ],
    'estado': [
        'Abandonado', 'En Curso','Graduado','Abandonado', 'En Curso','Graduado','Abandonado', 'Abandonado', 'En Curso',\
        'En Curso','Graduado','Graduado','Abandonado', 'Abandonado', 'En Curso','En Curso','Graduado','Graduado',\
        'Abandonado', 'En Curso','Graduado'
    ],
    'valor_educacion': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
})

# Y le pongo las mismas categorias que ya habia encontrado previamente.
valores_educacion.nombre = valores_educacion.nombre.astype((pd.api.types.CategoricalDtype(post_educacion.nombre.cat.categories)))
valores_educacion.estado = valores_educacion.estado.astype((pd.api.types.CategoricalDtype(post_educacion.estado.cat.categories)))

In [ ]:
# Hago el join para aplicar puntajes a las educaciones de los postulantes.
post_ed_total = pd.merge(post_ed_total, valores_educacion, how='inner', left_on=['nombre','estado'], right_on = ['nombre','estado'])

In [ ]:
# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_ed_total.groupby(['idpostulante'])['valor_educacion'].max().reset_index()

In [ ]:
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_ed_total = pd.merge(post_ed_total, max_post_educacion,  how='inner', left_on=['idpostulante','valor_educacion'], right_on = ['idpostulante','valor_educacion'])


In [ ]:
post_ed_total.drop_duplicates(inplace=True)

## Ahora exporto las educaciones del postulante

In [ ]:
post_ed_total.to_csv('../data/final_data/postulantes_educacion.csv', sep=',', encoding='utf-8')

## Sigo con los proximos DF

In [ ]:
post_gen_edad_total.head()

In [ ]:
#como supongo que con la edad no necesitare la fecha de nacimiento lo dropeo.
post_gen_edad_total.drop(['fechanacimiento'], axis=1, inplace=True)
post_gen_edad_total.head()

In [ ]:
#exporto al csv
post_gen_edad_total.to_csv('../data/final_data/postulantes_genero_y_edad.csv', sep=',', encoding='utf-8')

In [ ]:
vistas_total.head(20)

In [ ]:
vistas_total.drop_duplicates(subset=['idaviso','idpostulante'], keep='last')

In [ ]:
vistas_total.head(30)